# Comparación de Datos Originales vs Sintéticos - Dataset de Películas

Este notebook compara el dataset original de películas con datos sintéticos generados, analizando:
- Generación de datos sintéticos basados en distribuciones reales
- Comparación de estadísticas descriptivas
- Comparación de distribuciones
- Comparación de correlaciones
- Validación de similitud entre datasets

## 1. Importación de Librerías y Carga de Datos Originales

In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import warnings
from scipy import stats
from scipy.stats import ks_2samp, mannwhitneyu, chi2_contingency
warnings.filterwarnings('ignore')

# Configuración de estilo para visualizaciones
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Configuración de pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✓ Librerías importadas correctamente")
print(f"Versiones:")
print(f"  • Pandas: {pd.__version__}")
print(f"  • NumPy: {np.__version__}")
print(f"  • Matplotlib: {plt.matplotlib.__version__}")
print(f"  • Seaborn: {sns.__version__}")

In [ ]:
# Cargar el dataset original de películas
print("=" * 80)
print("CARGANDO DATASET ORIGINAL")
print("=" * 80)

with open('movies.json', 'r', encoding='utf-8') as f:
    movies_data = json.load(f)

# Convertir a DataFrame
df_original = pd.DataFrame(movies_data)

print(f"✓ Dataset original cargado exitosamente")
print(f"Dimensiones: {df_original.shape[0]} películas, {df_original.shape[1]} características")
print(f"\nPrimeras 5 filas:")
df_original.head()

## 2. Generación de Datos Sintéticos

Generaremos datos sintéticos que repliquen las características estadísticas del dataset original:
- Misma distribución de puntuaciones (puan)
- Misma distribución de popularidad (pop)
- Mismas proporciones de géneros
- Correlación similar entre variables

In [ ]:
print("=" * 80)
print("GENERANDO DATOS SINTÉTICOS")
print("=" * 80)

# Configurar semilla para reproducibilidad
np.random.seed(42)

# Número de películas sintéticas (mismo tamaño que el original)
n_synthetic = len(df_original)

# 1. Generar puntuaciones con distribución similar
print("\n1. Generando puntuaciones (puan)...")
puan_mean = df_original['puan'].mean()
puan_std = df_original['puan'].std()
puan_min = df_original['puan'].min()
puan_max = df_original['puan'].max()

# Generar con distribución normal truncada
puan_synthetic = np.random.normal(puan_mean, puan_std, n_synthetic)
puan_synthetic = np.clip(puan_synthetic, puan_min, puan_max)

print(f"   Original - Media: {puan_mean:.2f}, Std: {puan_std:.2f}")
print(f"   Sintético - Media: {puan_synthetic.mean():.2f}, Std: {puan_synthetic.std():.2f}")

# 2. Generar popularidad con distribución similar
print("\n2. Generando popularidad (pop)...")
pop_mean = df_original['pop'].mean()
pop_std = df_original['pop'].std()
pop_min = df_original['pop'].min()
pop_max = df_original['pop'].max()

# Generar con distribución normal truncada y redondear a enteros
pop_synthetic = np.random.normal(pop_mean, pop_std, n_synthetic)
pop_synthetic = np.clip(pop_synthetic, pop_min, pop_max)
pop_synthetic = np.round(pop_synthetic).astype(int)

print(f"   Original - Media: {pop_mean:.2f}, Std: {pop_std:.2f}")
print(f"   Sintético - Media: {pop_synthetic.mean():.2f}, Std: {pop_synthetic.std():.2f}")

# 3. Generar correlación entre puan y pop
print("\n3. Ajustando correlación entre puan y pop...")
original_corr = df_original[['puan', 'pop']].corr().iloc[0, 1]

# Crear matriz de covarianza
cov_matrix = np.array([
    [puan_std**2, original_corr * puan_std * pop_std],
    [original_corr * puan_std * pop_std, pop_std**2]
])

# Generar datos correlacionados
data_correlated = np.random.multivariate_normal(
    [puan_mean, pop_mean], 
    cov_matrix, 
    n_synthetic
)

puan_synthetic = np.clip(data_correlated[:, 0], puan_min, puan_max)
pop_synthetic = np.clip(data_correlated[:, 1], pop_min, pop_max)
pop_synthetic = np.round(pop_synthetic).astype(int)

synthetic_corr = np.corrcoef(puan_synthetic, pop_synthetic)[0, 1]
print(f"   Original - Correlación: {original_corr:.4f}")
print(f"   Sintético - Correlación: {synthetic_corr:.4f}")

# 4. Generar géneros con distribución similar
print("\n4. Generando géneros...")
genre1_probs = df_original['genre_1'].value_counts(normalize=True)
genre2_probs = df_original['genre_2'].value_counts(normalize=True)

genre1_synthetic = np.random.choice(
    genre1_probs.index, 
    size=n_synthetic, 
    p=genre1_probs.values
)

genre2_synthetic = np.random.choice(
    genre2_probs.index, 
    size=n_synthetic, 
    p=genre2_probs.values
)

print(f"   Géneros primarios únicos: {len(np.unique(genre1_synthetic))}")
print(f"   Géneros secundarios únicos: {len(np.unique(genre2_synthetic))}")

# 5. Generar IDs y nombres
print("\n5. Generando IDs y nombres...")
ids_synthetic = np.arange(10000, 10000 + n_synthetic)
names_synthetic = [f"Synthetic Movie {i}" for i in range(1, n_synthetic + 1)]

# 6. Generar descripciones sintéticas
print("\n6. Generando descripciones...")
descriptions_synthetic = [
    f"This is a synthetic {g1}/{g2} movie with rating {p:.1f} and popularity {pop}."
    for g1, g2, p, pop in zip(genre1_synthetic, genre2_synthetic, puan_synthetic, pop_synthetic)
]

# Crear DataFrame sintético
df_synthetic = pd.DataFrame({
    'ID': ids_synthetic,
    'name': names_synthetic,
    'puan': puan_synthetic,
    'genre_1': genre1_synthetic,
    'genre_2': genre2_synthetic,
    'pop': pop_synthetic,
    'description': descriptions_synthetic
})

print(f"\n✓ Dataset sintético generado exitosamente")
print(f"Dimensiones: {df_synthetic.shape[0]} películas, {df_synthetic.shape[1]} características")

# Guardar datos sintéticos
df_synthetic.to_json('movies_synthetic.json', orient='records', indent=2)
print(f"\n✓ Datos sintéticos guardados en 'movies_synthetic.json'")

## 3. Comparación de Estadísticas Descriptivas

Compararemos las estadísticas básicas entre el dataset original y el sintético.

In [ ]:
print("=" * 80)
print("COMPARACIÓN DE ESTADÍSTICAS DESCRIPTIVAS")
print("=" * 80)

# Función para calcular estadísticas
def calculate_stats(df, label):
    stats_dict = {
        'Dataset': label,
        'n': len(df),
        'puan_mean': df['puan'].mean(),
        'puan_std': df['puan'].std(),
        'puan_min': df['puan'].min(),
        'puan_max': df['puan'].max(),
        'puan_median': df['puan'].median(),
        'pop_mean': df['pop'].mean(),
        'pop_std': df['pop'].std(),
        'pop_min': df['pop'].min(),
        'pop_max': df['pop'].max(),
        'pop_median': df['pop'].median(),
        'correlation': df[['puan', 'pop']].corr().iloc[0, 1]
    }
    return stats_dict

# Calcular estadísticas para ambos datasets
stats_original = calculate_stats(df_original, 'Original')
stats_synthetic = calculate_stats(df_synthetic, 'Sintético')

# Crear DataFrame comparativo
comparison_df = pd.DataFrame([stats_original, stats_synthetic])

# Calcular diferencias porcentuales
diff_row = {
    'Dataset': 'Diferencia %',
    'n': 0,
    'puan_mean': abs(stats_original['puan_mean'] - stats_synthetic['puan_mean']) / stats_original['puan_mean'] * 100,
    'puan_std': abs(stats_original['puan_std'] - stats_synthetic['puan_std']) / stats_original['puan_std'] * 100,
    'puan_min': abs(stats_original['puan_min'] - stats_synthetic['puan_min']) / stats_original['puan_min'] * 100,
    'puan_max': abs(stats_original['puan_max'] - stats_synthetic['puan_max']) / stats_original['puan_max'] * 100,
    'puan_median': abs(stats_original['puan_median'] - stats_synthetic['puan_median']) / stats_original['puan_median'] * 100,
    'pop_mean': abs(stats_original['pop_mean'] - stats_synthetic['pop_mean']) / stats_original['pop_mean'] * 100,
    'pop_std': abs(stats_original['pop_std'] - stats_synthetic['pop_std']) / stats_original['pop_std'] * 100,
    'pop_min': abs(stats_original['pop_min'] - stats_synthetic['pop_min']) / abs(stats_original['pop_min']) * 100 if stats_original['pop_min'] != 0 else 0,
    'pop_max': abs(stats_original['pop_max'] - stats_synthetic['pop_max']) / stats_original['pop_max'] * 100,
    'pop_median': abs(stats_original['pop_median'] - stats_synthetic['pop_median']) / stats_original['pop_median'] * 100,
    'correlation': abs(stats_original['correlation'] - stats_synthetic['correlation']) / abs(stats_original['correlation']) * 100
}

comparison_df = pd.concat([comparison_df, pd.DataFrame([diff_row])], ignore_index=True)

# Formatear para visualización
print("\nCOMPARACIÓN NUMÉRICA:")
print("=" * 80)
display_df = comparison_df.copy()
for col in display_df.columns:
    if col != 'Dataset':
        display_df[col] = display_df[col].apply(lambda x: f"{x:.4f}")

print(display_df.to_string(index=False))

# Resumen de calidad
print("\n" + "=" * 80)
print("EVALUACIÓN DE SIMILITUD")
print("=" * 80)

# Calcular promedio de diferencias porcentuales
avg_diff = np.mean([
    diff_row['puan_mean'], diff_row['puan_std'], diff_row['puan_median'],
    diff_row['pop_mean'], diff_row['pop_std'], diff_row['pop_median'],
    diff_row['correlation']
])

print(f"\nDiferencia promedio: {avg_diff:.2f}%")

if avg_diff < 1:
    print("✓ EXCELENTE: Los datos sintéticos son muy similares al original")
elif avg_diff < 5:
    print("✓ BUENO: Los datos sintéticos son similares al original")
elif avg_diff < 10:
    print("⚠ ACEPTABLE: Los datos sintéticos son moderadamente similares")
else:
    print("❌ POBRE: Los datos sintéticos difieren significativamente")

print(f"\nDiferencias clave:")
print(f"  • Puntuación media: {diff_row['puan_mean']:.2f}%")
print(f"  • Popularidad media: {diff_row['pop_mean']:.2f}%")
print(f"  • Correlación: {diff_row['correlation']:.2f}%")

## 4. Comparación Visual de Distribuciones

Visualizaremos las distribuciones de puntuación y popularidad para ambos datasets.

In [ ]:
# Comparación visual de distribuciones
fig, axes = plt.subplots(2, 3, figsize=(20, 12))

# 1. Histograma de Puntuación
axes[0, 0].hist(df_original['puan'], bins=50, alpha=0.6, label='Original', 
                color='skyblue', edgecolor='black', density=True)
axes[0, 0].hist(df_synthetic['puan'], bins=50, alpha=0.6, label='Sintético', 
                color='salmon', edgecolor='black', density=True)
axes[0, 0].set_xlabel('Puntuación', fontsize=12)
axes[0, 0].set_ylabel('Densidad', fontsize=12)
axes[0, 0].set_title('Distribución de Puntuaciones', fontsize=14, fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. Box Plot de Puntuación
bp_data = [df_original['puan'], df_synthetic['puan']]
bp = axes[0, 1].boxplot(bp_data, labels=['Original', 'Sintético'], patch_artist=True,
                         medianprops=dict(color='red', linewidth=2))
bp['boxes'][0].set_facecolor('lightblue')
bp['boxes'][1].set_facecolor('lightcoral')
axes[0, 1].set_ylabel('Puntuación', fontsize=12)
axes[0, 1].set_title('Box Plot - Puntuaciones', fontsize=14, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3, axis='y')

# 3. Q-Q Plot de Puntuación
from scipy.stats import probplot
probplot(df_original['puan'], dist="norm", plot=axes[0, 2])
axes[0, 2].set_title('Q-Q Plot - Original (Puntuación)', fontsize=14, fontweight='bold')
axes[0, 2].grid(True, alpha=0.3)

# 4. Histograma de Popularidad
axes[1, 0].hist(df_original['pop'], bins=50, alpha=0.6, label='Original', 
                color='lightgreen', edgecolor='black', density=True)
axes[1, 0].hist(df_synthetic['pop'], bins=50, alpha=0.6, label='Sintético', 
                color='orchid', edgecolor='black', density=True)
axes[1, 0].set_xlabel('Popularidad', fontsize=12)
axes[1, 0].set_ylabel('Densidad', fontsize=12)
axes[1, 0].set_title('Distribución de Popularidad', fontsize=14, fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# 5. Box Plot de Popularidad
bp_data = [df_original['pop'], df_synthetic['pop']]
bp = axes[1, 1].boxplot(bp_data, labels=['Original', 'Sintético'], patch_artist=True,
                         medianprops=dict(color='red', linewidth=2))
bp['boxes'][0].set_facecolor('lightgreen')
bp['boxes'][1].set_facecolor('plum')
axes[1, 1].set_ylabel('Popularidad', fontsize=12)
axes[1, 1].set_title('Box Plot - Popularidad', fontsize=14, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3, axis='y')

# 6. Q-Q Plot de Popularidad
probplot(df_original['pop'], dist="norm", plot=axes[1, 2])
axes[1, 2].set_title('Q-Q Plot - Original (Popularidad)', fontsize=14, fontweight='bold')
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Interpretación:")
print("  • Los histogramas superpuestos muestran qué tan bien coinciden las distribuciones")
print("  • Los box plots permiten comparar mediana, cuartiles y outliers")
print("  • Los Q-Q plots evalúan la normalidad de las distribuciones originales")

## 5. Pruebas Estadísticas de Similitud

Realizaremos pruebas estadísticas para validar que las distribuciones son similares:
- **Test de Kolmogorov-Smirnov**: Compara distribuciones
- **Test de Mann-Whitney U**: Compara medianas
- **Test Chi-cuadrado**: Compara distribuciones de géneros

In [ ]:
print("=" * 80)
print("PRUEBAS ESTADÍSTICAS DE SIMILITUD")
print("=" * 80)

# Función para interpretar p-value (H0: las distribuciones son iguales)
def interpret_similarity_test(p_value, alpha=0.05):
    if p_value > alpha:
        return "✓ NO RECHAZAR H₀", "Las distribuciones SON similares"
    else:
        return "❌ RECHAZAR H₀", "Las distribuciones NO son similares"

# ============================================================================
# 1. Test de Kolmogorov-Smirnov para Puntuación
# ============================================================================
print("\n" + "=" * 80)
print("1. TEST DE KOLMOGOROV-SMIRNOV - PUNTUACIÓN")
print("=" * 80)
print("H₀: Las distribuciones de puntuación son iguales")
print("H₁: Las distribuciones de puntuación son diferentes")

ks_stat_puan, ks_p_puan = ks_2samp(df_original['puan'], df_synthetic['puan'])
decision, interpretation = interpret_similarity_test(ks_p_puan)

print(f"\nEstadístico KS: {ks_stat_puan:.6f}")
print(f"P-valor: {ks_p_puan:.6f}")
print(f"Decisión (α=0.05): {decision}")
print(f"Interpretación: {interpretation}")

# ============================================================================
# 2. Test de Kolmogorov-Smirnov para Popularidad
# ============================================================================
print("\n\n" + "=" * 80)
print("2. TEST DE KOLMOGOROV-SMIRNOV - POPULARIDAD")
print("=" * 80)
print("H₀: Las distribuciones de popularidad son iguales")
print("H₁: Las distribuciones de popularidad son diferentes")

ks_stat_pop, ks_p_pop = ks_2samp(df_original['pop'], df_synthetic['pop'])
decision, interpretation = interpret_similarity_test(ks_p_pop)

print(f"\nEstadístico KS: {ks_stat_pop:.6f}")
print(f"P-valor: {ks_p_pop:.6f}")
print(f"Decisión (α=0.05): {decision}")
print(f"Interpretación: {interpretation}")

# ============================================================================
# 3. Test de Mann-Whitney U para Puntuación
# ============================================================================
print("\n\n" + "=" * 80)
print("3. TEST DE MANN-WHITNEY U - PUNTUACIÓN")
print("=" * 80)
print("H₀: Las medianas de puntuación son iguales")
print("H₁: Las medianas de puntuación son diferentes")

mw_stat_puan, mw_p_puan = mannwhitneyu(df_original['puan'], df_synthetic['puan'], alternative='two-sided')
decision, interpretation = interpret_similarity_test(mw_p_puan)

print(f"\nEstadístico U: {mw_stat_puan:.0f}")
print(f"P-valor: {mw_p_puan:.6f}")
print(f"Decisión (α=0.05): {decision}")
print(f"Interpretación: {interpretation}")

# ============================================================================
# 4. Test de Mann-Whitney U para Popularidad
# ============================================================================
print("\n\n" + "=" * 80)
print("4. TEST DE MANN-WHITNEY U - POPULARIDAD")
print("=" * 80)
print("H₀: Las medianas de popularidad son iguales")
print("H₁: Las medianas de popularidad son diferentes")

mw_stat_pop, mw_p_pop = mannwhitneyu(df_original['pop'], df_synthetic['pop'], alternative='two-sided')
decision, interpretation = interpret_similarity_test(mw_p_pop)

print(f"\nEstadístico U: {mw_stat_pop:.0f}")
print(f"P-valor: {mw_p_pop:.6f}")
print(f"Decisión (α=0.05): {decision}")
print(f"Interpretación: {interpretation}")

# ============================================================================
# 5. Test Chi-cuadrado para Géneros Primarios
# ============================================================================
print("\n\n" + "=" * 80)
print("5. TEST CHI-CUADRADO - DISTRIBUCIÓN DE GÉNEROS PRIMARIOS")
print("=" * 80)
print("H₀: Las distribuciones de géneros son independientes del dataset")
print("H₁: Las distribuciones de géneros dependen del dataset")

# Crear tabla de contingencia para top 10 géneros
top_genres = df_original['genre_1'].value_counts().head(10).index
orig_counts = df_original[df_original['genre_1'].isin(top_genres)]['genre_1'].value_counts().sort_index()
synt_counts = df_synthetic[df_synthetic['genre_1'].isin(top_genres)]['genre_1'].value_counts().sort_index()

# Alinear índices
all_genres = sorted(set(orig_counts.index) | set(synt_counts.index))
orig_aligned = [orig_counts.get(g, 0) for g in all_genres]
synt_aligned = [synt_counts.get(g, 0) for g in all_genres]

contingency_table = np.array([orig_aligned, synt_aligned])

chi2_stat, chi2_p, dof, expected = chi2_contingency(contingency_table)
decision, interpretation = interpret_similarity_test(chi2_p)

print(f"\nEstadístico χ²: {chi2_stat:.6f}")
print(f"Grados de libertad: {dof}")
print(f"P-valor: {chi2_p:.6f}")
print(f"Decisión (α=0.05): {decision}")
print(f"Interpretación: {interpretation}")

# ============================================================================
# RESUMEN
# ============================================================================
print("\n\n" + "=" * 80)
print("RESUMEN DE PRUEBAS DE SIMILITUD")
print("=" * 80)

results = pd.DataFrame({
    'Prueba': [
        'KS - Puntuación',
        'KS - Popularidad',
        'Mann-Whitney - Puntuación',
        'Mann-Whitney - Popularidad',
        'Chi-cuadrado - Géneros'
    ],
    'P-valor': [ks_p_puan, ks_p_pop, mw_p_puan, mw_p_pop, chi2_p],
    'Resultado': [
        'Similar' if ks_p_puan > 0.05 else 'Diferente',
        'Similar' if ks_p_pop > 0.05 else 'Diferente',
        'Similar' if mw_p_puan > 0.05 else 'Diferente',
        'Similar' if mw_p_pop > 0.05 else 'Diferente',
        'Similar' if chi2_p > 0.05 else 'Diferente'
    ]
})

print("\n" + results.to_string(index=False))

# Contar tests que pasaron
tests_passed = sum([
    ks_p_puan > 0.05,
    ks_p_pop > 0.05,
    mw_p_puan > 0.05,
    mw_p_pop > 0.05,
    chi2_p > 0.05
])

print(f"\n\nTests que indican similitud: {tests_passed}/5 ({tests_passed/5*100:.0f}%)")

if tests_passed >= 4:
    print("\n✓ CONCLUSIÓN: Los datos sintéticos replican exitosamente el dataset original")
elif tests_passed >= 3:
    print("\n⚠ CONCLUSIÓN: Los datos sintéticos son razonablemente similares al original")
else:
    print("\n❌ CONCLUSIÓN: Los datos sintéticos difieren significativamente del original")

print("\n" + "=" * 80)

## 6. Comparación de Correlaciones

Analizaremos en detalle la correlación entre puntuación y popularidad en ambos datasets.

In [ ]:
print("=" * 80)
print("COMPARACIÓN DE CORRELACIONES")
print("=" * 80)

# Calcular matrices de correlación
corr_original = df_original[['puan', 'pop']].corr()
corr_synthetic = df_synthetic[['puan', 'pop']].corr()

print("\nMATRIZ DE CORRELACIÓN - ORIGINAL:")
print(corr_original)

print("\n\nMATRIZ DE CORRELACIÓN - SINTÉTICO:")
print(corr_synthetic)

# Diferencia en correlación
corr_diff = abs(corr_original.iloc[0, 1] - corr_synthetic.iloc[0, 1])
corr_diff_pct = corr_diff / abs(corr_original.iloc[0, 1]) * 100

print(f"\n\nDIFERENCIA EN CORRELACIÓN:")
print(f"  Original:  {corr_original.iloc[0, 1]:.6f}")
print(f"  Sintético: {corr_synthetic.iloc[0, 1]:.6f}")
print(f"  Diferencia absoluta: {corr_diff:.6f}")
print(f"  Diferencia porcentual: {corr_diff_pct:.2f}%")

# Visualización de correlaciones
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# 1. Mapa de calor - Original
sns.heatmap(corr_original, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=2, cbar_kws={"shrink": 0.8}, 
            fmt='.4f', ax=axes[0], vmin=-1, vmax=1)
axes[0].set_title('Correlación - Original', fontsize=14, fontweight='bold')

# 2. Mapa de calor - Sintético
sns.heatmap(corr_synthetic, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=2, cbar_kws={"shrink": 0.8}, 
            fmt='.4f', ax=axes[1], vmin=-1, vmax=1)
axes[1].set_title('Correlación - Sintético', fontsize=14, fontweight='bold')

# 3. Scatter plots superpuestos
axes[2].scatter(df_original['puan'], df_original['pop'], alpha=0.4, 
               color='blue', s=20, label='Original', edgecolors='none')
axes[2].scatter(df_synthetic['puan'], df_synthetic['pop'], alpha=0.4, 
               color='red', s=20, label='Sintético', edgecolors='none')

# Líneas de tendencia
z_orig = np.polyfit(df_original['puan'], df_original['pop'], 1)
p_orig = np.poly1d(z_orig)
z_synt = np.polyfit(df_synthetic['puan'], df_synthetic['pop'], 1)
p_synt = np.poly1d(z_synt)

x_range = np.linspace(df_original['puan'].min(), df_original['puan'].max(), 100)
axes[2].plot(x_range, p_orig(x_range), "b--", linewidth=2, label=f'Tendencia Original (r={corr_original.iloc[0,1]:.3f})')
axes[2].plot(x_range, p_synt(x_range), "r--", linewidth=2, label=f'Tendencia Sintético (r={corr_synthetic.iloc[0,1]:.3f})')

axes[2].set_xlabel('Puntuación (puan)', fontsize=12)
axes[2].set_ylabel('Popularidad (pop)', fontsize=12)
axes[2].set_title('Relación Puntuación-Popularidad', fontsize=14, fontweight='bold')
axes[2].legend(loc='best')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

if corr_diff_pct < 5:
    print("\n✓ Las correlaciones son muy similares")
elif corr_diff_pct < 10:
    print("\n⚠ Las correlaciones son moderadamente similares")
else:
    print("\n❌ Las correlaciones difieren significativamente")

## 7. Comparación de Distribución de Géneros

Analizaremos si los géneros se distribuyen de forma similar en ambos datasets.

In [ ]:
print("=" * 80)
print("COMPARACIÓN DE DISTRIBUCIÓN DE GÉNEROS")
print("=" * 80)

# Obtener top 15 géneros de ambos datasets
top_n = 15
genre1_orig = df_original['genre_1'].value_counts().head(top_n)
genre1_synt = df_synthetic['genre_1'].value_counts().head(top_n)

# Crear DataFrame comparativo
all_genres = sorted(set(genre1_orig.index) | set(genre1_synt.index))
comparison_genres = pd.DataFrame({
    'Género': all_genres,
    'Original': [genre1_orig.get(g, 0) for g in all_genres],
    'Sintético': [genre1_synt.get(g, 0) for g in all_genres]
})

comparison_genres['Diferencia'] = comparison_genres['Original'] - comparison_genres['Sintético']
comparison_genres['Diferencia_%'] = (comparison_genres['Diferencia'] / comparison_genres['Original'] * 100).round(2)
comparison_genres = comparison_genres.sort_values('Original', ascending=False)

print("\nCOMPARACIÓN DE FRECUENCIAS - GÉNEROS PRIMARIOS (Top 15):")
print("=" * 80)
print(comparison_genres.to_string(index=False))

# Calcular error promedio
avg_error = abs(comparison_genres['Diferencia_%']).mean()
print(f"\n\nError promedio en distribución de géneros: {avg_error:.2f}%")

# Visualización
fig, axes = plt.subplots(2, 2, figsize=(18, 12))

# 1. Comparación de barras - Top 15 géneros primarios
x_pos = np.arange(len(comparison_genres))
width = 0.35

axes[0, 0].barh(x_pos - width/2, comparison_genres['Original'], width, 
                label='Original', color='steelblue', alpha=0.8)
axes[0, 0].barh(x_pos + width/2, comparison_genres['Sintético'], width, 
                label='Sintético', color='coral', alpha=0.8)
axes[0, 0].set_yticks(x_pos)
axes[0, 0].set_yticklabels(comparison_genres['Género'], fontsize=9)
axes[0, 0].set_xlabel('Frecuencia', fontsize=12)
axes[0, 0].set_title('Distribución de Géneros Primarios - Comparación', fontsize=14, fontweight='bold')
axes[0, 0].legend()
axes[0, 0].invert_yaxis()
axes[0, 0].grid(True, alpha=0.3, axis='x')

# 2. Scatter plot de frecuencias
axes[0, 1].scatter(comparison_genres['Original'], comparison_genres['Sintético'], 
                   s=100, alpha=0.6, color='purple', edgecolors='black')
# Línea de identidad perfecta
max_val = max(comparison_genres['Original'].max(), comparison_genres['Sintético'].max())
axes[0, 1].plot([0, max_val], [0, max_val], 'r--', linewidth=2, label='Identidad perfecta')
axes[0, 1].set_xlabel('Frecuencia Original', fontsize=12)
axes[0, 1].set_ylabel('Frecuencia Sintética', fontsize=12)
axes[0, 1].set_title('Correlación de Frecuencias por Género', fontsize=14, fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Añadir etiquetas a los puntos
for idx, row in comparison_genres.iterrows():
    if row['Original'] > comparison_genres['Original'].mean():
        axes[0, 1].annotate(row['Género'], 
                           (row['Original'], row['Sintético']), 
                           fontsize=8, alpha=0.7)

# 3. Gráfico de error porcentual
axes[1, 0].barh(range(len(comparison_genres)), abs(comparison_genres['Diferencia_%']), 
                color='orange', alpha=0.7)
axes[1, 0].set_yticks(range(len(comparison_genres)))
axes[1, 0].set_yticklabels(comparison_genres['Género'], fontsize=9)
axes[1, 0].set_xlabel('Error Porcentual (%)', fontsize=12)
axes[1, 0].set_title('Error en Distribución por Género', fontsize=14, fontweight='bold')
axes[1, 0].axvline(10, color='red', linestyle='--', linewidth=2, label='Umbral 10%')
axes[1, 0].legend()
axes[1, 0].invert_yaxis()
axes[1, 0].grid(True, alpha=0.3, axis='x')

# 4. Gráfico de pastel comparativo (solo top 5)
top_5_genres = comparison_genres.head(5)
others_orig = df_original['genre_1'].value_counts()[top_n:].sum()
others_synt = df_synthetic['genre_1'].value_counts()[top_n:].sum()

labels = list(top_5_genres['Género']) + ['Otros']
orig_values = list(top_5_genres['Original']) + [others_orig]
synt_values = list(top_5_genres['Sintético']) + [others_synt]

axes[1, 1].pie([sum(orig_values), sum(synt_values)], 
               labels=['Original', 'Sintético'],
               autopct='%1.1f%%', startangle=90, colors=['steelblue', 'coral'])
axes[1, 1].set_title('Proporción Total de Géneros', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

# Evaluación de similitud
print("\n" + "=" * 80)
print("EVALUACIÓN DE SIMILITUD EN GÉNEROS")
print("=" * 80)

if avg_error < 5:
    print(f"\n✓ EXCELENTE: Error promedio {avg_error:.2f}% - Distribuciones muy similares")
elif avg_error < 10:
    print(f"\n✓ BUENO: Error promedio {avg_error:.2f}% - Distribuciones similares")
elif avg_error < 20:
    print(f"\n⚠ ACEPTABLE: Error promedio {avg_error:.2f}% - Distribuciones moderadamente similares")
else:
    print(f"\n❌ POBRE: Error promedio {avg_error:.2f}% - Distribuciones diferentes")

# Géneros con mayor diferencia
print(f"\nGéneros con mayor diferencia:")
top_diff = comparison_genres.nlargest(3, 'Diferencia_%')[['Género', 'Original', 'Sintético', 'Diferencia_%']]
for idx, row in top_diff.iterrows():
    print(f"  • {row['Género']}: {row['Diferencia_%']:.2f}% de diferencia")

## 8. Análisis de Momentos Estadísticos

Compararemos los momentos estadísticos (asimetría y curtosis) de ambas distribuciones.

In [ ]:
from scipy.stats import skew, kurtosis

print("=" * 80)
print("ANÁLISIS DE MOMENTOS ESTADÍSTICOS")
print("=" * 80)

# Calcular momentos para ambos datasets
moments_data = {
    'Métrica': ['Asimetría (Skewness)', 'Curtosis (Kurtosis)'],
    'Original_puan': [
        skew(df_original['puan']),
        kurtosis(df_original['puan'])
    ],
    'Sintético_puan': [
        skew(df_synthetic['puan']),
        kurtosis(df_synthetic['puan'])
    ],
    'Original_pop': [
        skew(df_original['pop']),
        kurtosis(df_original['pop'])
    ],
    'Sintético_pop': [
        skew(df_synthetic['pop']),
        kurtosis(df_synthetic['pop'])
    ]
}

moments_df = pd.DataFrame(moments_data)

# Calcular diferencias
moments_df['Diff_puan_%'] = abs(moments_df['Original_puan'] - moments_df['Sintético_puan']) / abs(moments_df['Original_puan']) * 100
moments_df['Diff_pop_%'] = abs(moments_df['Original_pop'] - moments_df['Sintético_pop']) / abs(moments_df['Original_pop']) * 100

print("\nCOMPARACIÓN DE MOMENTOS ESTADÍSTICOS:")
print("=" * 80)

# Formatear para mejor visualización
display_moments = moments_df.copy()
for col in display_moments.columns:
    if col != 'Métrica':
        display_moments[col] = display_moments[col].apply(lambda x: f"{x:.6f}")

print(display_moments.to_string(index=False))

# Interpretación de asimetría
print("\n\n" + "=" * 80)
print("INTERPRETACIÓN DE ASIMETRÍA (SKEWNESS)")
print("=" * 80)

def interpret_skewness(skew_value):
    if abs(skew_value) < 0.5:
        return "Aproximadamente simétrica"
    elif skew_value < -0.5:
        return "Asimétrica negativa (cola izquierda)"
    else:
        return "Asimétrica positiva (cola derecha)"

print("\nPUNTUACIÓN (puan):")
print(f"  Original:  {skew(df_original['puan']):.4f} - {interpret_skewness(skew(df_original['puan']))}")
print(f"  Sintético: {skew(df_synthetic['puan']):.4f} - {interpret_skewness(skew(df_synthetic['puan']))}")

print("\nPOPULARIDAD (pop):")
print(f"  Original:  {skew(df_original['pop']):.4f} - {interpret_skewness(skew(df_original['pop']))}")
print(f"  Sintético: {skew(df_synthetic['pop']):.4f} - {interpret_skewness(skew(df_synthetic['pop']))}")

# Interpretación de curtosis
print("\n\n" + "=" * 80)
print("INTERPRETACIÓN DE CURTOSIS (KURTOSIS)")
print("=" * 80)

def interpret_kurtosis(kurt_value):
    if abs(kurt_value) < 0.5:
        return "Mesocúrtica (normal)"
    elif kurt_value < -0.5:
        return "Platicúrtica (más plana que normal)"
    else:
        return "Leptocúrtica (más puntiaguda que normal)"

print("\nPUNTUACIÓN (puan):")
print(f"  Original:  {kurtosis(df_original['puan']):.4f} - {interpret_kurtosis(kurtosis(df_original['puan']))}")
print(f"  Sintético: {kurtosis(df_synthetic['puan']):.4f} - {interpret_kurtosis(kurtosis(df_synthetic['puan']))}")

print("\nPOPULARIDAD (pop):")
print(f"  Original:  {kurtosis(df_original['pop']):.4f} - {interpret_kurtosis(kurtosis(df_original['pop']))}")
print(f"  Sintético: {kurtosis(df_synthetic['pop']):.4f} - {interpret_kurtosis(kurtosis(df_synthetic['pop']))}")

# Visualización de momentos
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Gráfico de comparación de asimetría
categories = ['Puan', 'Pop']
orig_skew = [skew(df_original['puan']), skew(df_original['pop'])]
synt_skew = [skew(df_synthetic['puan']), skew(df_synthetic['pop'])]

x_pos = np.arange(len(categories))
width = 0.35

axes[0].bar(x_pos - width/2, orig_skew, width, label='Original', color='steelblue', alpha=0.8)
axes[0].bar(x_pos + width/2, synt_skew, width, label='Sintético', color='coral', alpha=0.8)
axes[0].set_xlabel('Variable', fontsize=12)
axes[0].set_ylabel('Asimetría', fontsize=12)
axes[0].set_title('Comparación de Asimetría', fontsize=14, fontweight='bold')
axes[0].set_xticks(x_pos)
axes[0].set_xticklabels(categories)
axes[0].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')

# Gráfico de comparación de curtosis
orig_kurt = [kurtosis(df_original['puan']), kurtosis(df_original['pop'])]
synt_kurt = [kurtosis(df_synthetic['puan']), kurtosis(df_synthetic['pop'])]

axes[1].bar(x_pos - width/2, orig_kurt, width, label='Original', color='steelblue', alpha=0.8)
axes[1].bar(x_pos + width/2, synt_kurt, width, label='Sintético', color='coral', alpha=0.8)
axes[1].set_xlabel('Variable', fontsize=12)
axes[1].set_ylabel('Curtosis', fontsize=12)
axes[1].set_title('Comparación de Curtosis', fontsize=14, fontweight='bold')
axes[1].set_xticks(x_pos)
axes[1].set_xticklabels(categories)
axes[1].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Resumen de similitud en momentos
avg_diff_moments = (
    abs(skew(df_original['puan']) - skew(df_synthetic['puan'])) / abs(skew(df_original['puan'])) * 100 +
    abs(kurtosis(df_original['puan']) - kurtosis(df_synthetic['puan'])) / abs(kurtosis(df_original['puan'])) * 100 +
    abs(skew(df_original['pop']) - skew(df_synthetic['pop'])) / abs(skew(df_original['pop'])) * 100 +
    abs(kurtosis(df_original['pop']) - kurtosis(df_synthetic['pop'])) / abs(kurtosis(df_original['pop'])) * 100
) / 4

print("\n" + "=" * 80)
print(f"Diferencia promedio en momentos: {avg_diff_moments:.2f}%")

if avg_diff_moments < 10:
    print("✓ Los momentos estadísticos son muy similares")
elif avg_diff_moments < 25:
    print("⚠ Los momentos estadísticos son moderadamente similares")
else:
    print("❌ Los momentos estadísticos difieren significativamente")

## 9. Análisis de Cuantiles

Compararemos los cuantiles para verificar que ambas distribuciones tienen rangos similares.

In [ ]:
print("=" * 80)
print("ANÁLISIS DE CUANTILES")
print("=" * 80)

# Definir cuantiles a analizar
quantiles = [0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95]

# Calcular cuantiles para puntuación
quantiles_puan_orig = df_original['puan'].quantile(quantiles)
quantiles_puan_synt = df_synthetic['puan'].quantile(quantiles)

# Calcular cuantiles para popularidad
quantiles_pop_orig = df_original['pop'].quantile(quantiles)
quantiles_pop_synt = df_synthetic['pop'].quantile(quantiles)

# Crear DataFrame comparativo
quantiles_comparison = pd.DataFrame({
    'Cuantil': [f'{int(q*100)}%' for q in quantiles],
    'Puan_Original': quantiles_puan_orig.values,
    'Puan_Sintético': quantiles_puan_synt.values,
    'Puan_Diff': abs(quantiles_puan_orig.values - quantiles_puan_synt.values),
    'Pop_Original': quantiles_pop_orig.values,
    'Pop_Sintético': quantiles_pop_synt.values,
    'Pop_Diff': abs(quantiles_pop_orig.values - quantiles_pop_synt.values)
})

print("\nCOMPARACIÓN DE CUANTILES:")
print("=" * 80)
print(quantiles_comparison.to_string(index=False))

# Calcular diferencias promedio
avg_diff_puan = quantiles_comparison['Puan_Diff'].mean()
avg_diff_pop = quantiles_comparison['Pop_Diff'].mean()

print(f"\n\nDiferencia promedio en cuantiles:")
print(f"  • Puntuación: {avg_diff_puan:.4f}")
print(f"  • Popularidad: {avg_diff_pop:.4f}")

# Visualización de cuantiles
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Comparación de cuantiles - Puntuación
x_pos = np.arange(len(quantiles))
width = 0.35

axes[0, 0].bar(x_pos - width/2, quantiles_puan_orig.values, width, 
               label='Original', color='steelblue', alpha=0.8)
axes[0, 0].bar(x_pos + width/2, quantiles_puan_synt.values, width, 
               label='Sintético', color='coral', alpha=0.8)
axes[0, 0].set_xlabel('Cuantil', fontsize=12)
axes[0, 0].set_ylabel('Valor', fontsize=12)
axes[0, 0].set_title('Comparación de Cuantiles - Puntuación', fontsize=14, fontweight='bold')
axes[0, 0].set_xticks(x_pos)
axes[0, 0].set_xticklabels([f'{int(q*100)}%' for q in quantiles])
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3, axis='y')

# 2. Diferencias en cuantiles - Puntuación
axes[0, 1].bar(x_pos, quantiles_comparison['Puan_Diff'].values, 
               color='orange', alpha=0.8)
axes[0, 1].set_xlabel('Cuantil', fontsize=12)
axes[0, 1].set_ylabel('Diferencia Absoluta', fontsize=12)
axes[0, 1].set_title('Diferencias en Cuantiles - Puntuación', fontsize=14, fontweight='bold')
axes[0, 1].set_xticks(x_pos)
axes[0, 1].set_xticklabels([f'{int(q*100)}%' for q in quantiles])
axes[0, 1].axhline(y=avg_diff_puan, color='red', linestyle='--', 
                   linewidth=2, label=f'Promedio: {avg_diff_puan:.4f}')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3, axis='y')

# 3. Comparación de cuantiles - Popularidad
axes[1, 0].bar(x_pos - width/2, quantiles_pop_orig.values, width, 
               label='Original', color='lightgreen', alpha=0.8)
axes[1, 0].bar(x_pos + width/2, quantiles_pop_synt.values, width, 
               label='Sintético', color='orchid', alpha=0.8)
axes[1, 0].set_xlabel('Cuantil', fontsize=12)
axes[1, 0].set_ylabel('Valor', fontsize=12)
axes[1, 0].set_title('Comparación de Cuantiles - Popularidad', fontsize=14, fontweight='bold')
axes[1, 0].set_xticks(x_pos)
axes[1, 0].set_xticklabels([f'{int(q*100)}%' for q in quantiles])
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3, axis='y')

# 4. Diferencias en cuantiles - Popularidad
axes[1, 1].bar(x_pos, quantiles_comparison['Pop_Diff'].values, 
               color='purple', alpha=0.8)
axes[1, 1].set_xlabel('Cuantil', fontsize=12)
axes[1, 1].set_ylabel('Diferencia Absoluta', fontsize=12)
axes[1, 1].set_title('Diferencias en Cuantiles - Popularidad', fontsize=14, fontweight='bold')
axes[1, 1].set_xticks(x_pos)
axes[1, 1].set_xticklabels([f'{int(q*100)}%' for q in quantiles])
axes[1, 1].axhline(y=avg_diff_pop, color='red', linestyle='--', 
                   linewidth=2, label=f'Promedio: {avg_diff_pop:.4f}')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Q-Q plots para comparación directa
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Q-Q plot para puntuación
axes[0].scatter(quantiles_puan_orig.values, quantiles_puan_synt.values, 
                s=100, alpha=0.7, color='steelblue', edgecolors='black')
# Línea de identidad
min_val = min(quantiles_puan_orig.min(), quantiles_puan_synt.min())
max_val = max(quantiles_puan_orig.max(), quantiles_puan_synt.max())
axes[0].plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Identidad perfecta')
axes[0].set_xlabel('Cuantiles Original', fontsize=12)
axes[0].set_ylabel('Cuantiles Sintético', fontsize=12)
axes[0].set_title('Q-Q Plot - Puntuación', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Q-Q plot para popularidad
axes[1].scatter(quantiles_pop_orig.values, quantiles_pop_synt.values, 
                s=100, alpha=0.7, color='lightgreen', edgecolors='black')
# Línea de identidad
min_val = min(quantiles_pop_orig.min(), quantiles_pop_synt.min())
max_val = max(quantiles_pop_orig.max(), quantiles_pop_synt.max())
axes[1].plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Identidad perfecta')
axes[1].set_xlabel('Cuantiles Original', fontsize=12)
axes[1].set_ylabel('Cuantiles Sintético', fontsize=12)
axes[1].set_title('Q-Q Plot - Popularidad', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Interpretación:")
print("  • Los Q-Q plots muestran qué tan bien coinciden los cuantiles")
print("  • Puntos cerca de la línea roja indican buena similitud")
print("  • Desviaciones indican diferencias en las colas de las distribuciones")

## 10. Resumen Final y Conclusiones

Consolidaremos todos los resultados para evaluar la calidad de los datos sintéticos generados.

In [ ]:
print("=" * 80)
print("RESUMEN FINAL - COMPARACIÓN DATASET ORIGINAL VS SINTÉTICO")
print("=" * 80)

# Crear tabla resumen consolidada
summary_results = {
    'Aspecto': [
        '1. Tamaño del dataset',
        '2. Media puntuación',
        '3. Desv. Est. puntuación',
        '4. Media popularidad',
        '5. Desv. Est. popularidad',
        '6. Correlación puan-pop',
        '7. Asimetría puntuación',
        '8. Curtosis puntuación',
        '9. Asimetría popularidad',
        '10. Curtosis popularidad'
    ],
    'Original': [
        len(df_original),
        f"{df_original['puan'].mean():.4f}",
        f"{df_original['puan'].std():.4f}",
        f"{df_original['pop'].mean():.4f}",
        f"{df_original['pop'].std():.4f}",
        f"{df_original[['puan', 'pop']].corr().iloc[0,1]:.4f}",
        f"{skew(df_original['puan']):.4f}",
        f"{kurtosis(df_original['puan']):.4f}",
        f"{skew(df_original['pop']):.4f}",
        f"{kurtosis(df_original['pop']):.4f}"
    ],
    'Sintético': [
        len(df_synthetic),
        f"{df_synthetic['puan'].mean():.4f}",
        f"{df_synthetic['puan'].std():.4f}",
        f"{df_synthetic['pop'].mean():.4f}",
        f"{df_synthetic['pop'].std():.4f}",
        f"{df_synthetic[['puan', 'pop']].corr().iloc[0,1]:.4f}",
        f"{skew(df_synthetic['puan']):.4f}",
        f"{kurtosis(df_synthetic['puan']):.4f}",
        f"{skew(df_synthetic['pop']):.4f}",
        f"{kurtosis(df_synthetic['pop']):.4f}"
    ]
}

summary_df = pd.DataFrame(summary_results)
print("\n" + summary_df.to_string(index=False))

# Resumen de pruebas estadísticas
print("\n\n" + "=" * 80)
print("RESULTADOS DE PRUEBAS ESTADÍSTICAS")
print("=" * 80)

statistical_tests = pd.DataFrame({
    'Prueba': [
        'Kolmogorov-Smirnov (Puntuación)',
        'Kolmogorov-Smirnov (Popularidad)',
        'Mann-Whitney U (Puntuación)',
        'Mann-Whitney U (Popularidad)',
        'Chi-cuadrado (Géneros)'
    ],
    'P-valor': [
        f"{ks_p_puan:.6f}",
        f"{ks_p_pop:.6f}",
        f"{mw_p_puan:.6f}",
        f"{mw_p_pop:.6f}",
        f"{chi2_p:.6f}"
    ],
    'Resultado (α=0.05)': [
        '✓ Similar' if ks_p_puan > 0.05 else '❌ Diferente',
        '✓ Similar' if ks_p_pop > 0.05 else '❌ Diferente',
        '✓ Similar' if mw_p_puan > 0.05 else '❌ Diferente',
        '✓ Similar' if mw_p_pop > 0.05 else '❌ Diferente',
        '✓ Similar' if chi2_p > 0.05 else '❌ Diferente'
    ]
})

print("\n" + statistical_tests.to_string(index=False))

# Cálculo de puntuación de similitud global
tests_passed = sum([
    ks_p_puan > 0.05,
    ks_p_pop > 0.05,
    mw_p_puan > 0.05,
    mw_p_pop > 0.05,
    chi2_p > 0.05
])

similarity_score = (tests_passed / 5) * 100

print("\n\n" + "=" * 80)
print("PUNTUACIÓN DE SIMILITUD GLOBAL")
print("=" * 80)

print(f"\nTests estadísticos aprobados: {tests_passed}/5")
print(f"Puntuación de similitud: {similarity_score:.0f}%")

# Barra de progreso visual
bar_length = 50
filled_length = int(bar_length * tests_passed / 5)
bar = '█' * filled_length + '░' * (bar_length - filled_length)
print(f"\n[{bar}] {similarity_score:.0f}%")

# Evaluación final
print("\n" + "=" * 80)
print("EVALUACIÓN FINAL")
print("=" * 80)

if similarity_score >= 80:
    grade = "EXCELENTE ✓✓✓"
    color = "🟢"
elif similarity_score >= 60:
    grade = "BUENO ✓✓"
    color = "🟡"
elif similarity_score >= 40:
    grade = "ACEPTABLE ✓"
    color = "🟠"
else:
    grade = "INSUFICIENTE ✗"
    color = "🔴"

print(f"\n{color} Calificación: {grade}")
print(f"\nPuntuación final: {similarity_score:.0f}/100")

print("\n\n" + "=" * 80)
print("CONCLUSIONES PRINCIPALES")
print("=" * 80)

print("\n✓ FORTALEZAS:")
print("  1. Tamaño del dataset: Idéntico al original")
print("  2. Estadísticas descriptivas: Alta similitud en medias y desviaciones")
print("  3. Correlaciones: Se preserva la relación entre puntuación y popularidad")
print("  4. Distribución de géneros: Replica las proporciones originales")

print("\n⚠ CONSIDERACIONES:")
print("  1. Los datos sintéticos son generados, no reales")
print("  2. Pueden no capturar todas las complejidades del dataset original")
print("  3. La asimetría y curtosis pueden diferir levemente")
print("  4. Las combinaciones específicas de valores pueden ser artificiales")

print("\n📊 RECOMENDACIONES DE USO:")
if similarity_score >= 70:
    print("  • Los datos sintéticos son APTOS para:")
    print("    - Pruebas de algoritmos de recomendación")
    print("    - Validación de pipelines de procesamiento")
    print("    - Desarrollo y debugging de sistemas")
    print("    - Entrenamiento preliminar de modelos")
else:
    print("  • Los datos sintéticos tienen LIMITACIONES para:")
    print("    - Análisis de producción")
    print("    - Conclusiones sobre el dataset real")
    print("    - Se recomienda ajustar el proceso de generación")

print("\n\n" + "=" * 80)
print("ARCHIVOS GENERADOS")
print("=" * 80)
print("\n✓ movies_synthetic.json - Dataset sintético completo")
print(f"  Contiene {len(df_synthetic)} películas sintéticas")
print(f"  Tamaño del archivo: ~{len(df_synthetic) * 200 / 1024:.1f} KB (estimado)")

print("\n\n" + "=" * 80)
print("FIN DEL ANÁLISIS COMPARATIVO")
print("=" * 80)